In [1]:
ARTIFACTS_FOLDER = "./artifacts"
!rm -rf images
!rm -rf maps
!rm -rf lidar
!ln -s /home/bob/data/lyft_data/train_images images
!ln -s /home/bob/data/lyft_data/train_maps maps
!ln -s /home/bob/data/lyft_data/train_lidar lidar

In [2]:
%load_ext autoreload

from datetime import datetime
from functools import partial
import glob
from multiprocessing import Pool

# Disable multiprocesing for numpy/opencv. We already multiprocess ourselves, this would mean every subprocess produces
# even more threads which would lead to a lot of context switching, slowing things down a lot.
import os
os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import pandas as pd
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
import scipy
import scipy.ndimage
import scipy.special
from scipy.spatial.transform import Rotation as R
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix

In [3]:
def create_transformation_matrix_to_voxel_space(shape, voxel_size, offset):
    """
    Constructs a transformation matrix given an output voxel shape such that (0,0,0) ends up in the center.
    Voxel_size defines how large every voxel is in world coordinate, 
    (1,1,1) would be the same as Minecraft voxels.
    An offset per axis in world coordinates (metric) can be provided, 
    this is useful for Z (up-down) in lidar points.
    No rotation, only a scale and translation.
    """
    
    shape, voxel_size, offset = np.array(shape), np.array(voxel_size), np.array(offset)
    tm = np.eye(4, dtype=np.float32) #(4,4)
    translation = shape/2 + offset/voxel_size #(3,)
    tm = tm * np.array(np.hstack((1/voxel_size, [1]))) # (4,4)
    tm[:3, 3] = np.transpose(translation)
    return tm

def transform_points(points, transf_matrix):
    """
    Transform (3,N) or (4,N) points using transformation matrix.
    """
    if points.shape[0] not in [3,4]:
        raise Exception("Points input should be (3,N) or (4,N) shape, received {}".format(points.shape))
    return transf_matrix.dot(np.vstack((points[:3, :], np.ones(points.shape[1]))))[:3, :]

def car_to_voxel_coords(points, shape, voxel_size, z_offset=0):
    if len(shape) != 3:
        raise Exception("Voxel volume shape should be 3 dimensions (x,y,z)")
        
    if len(points.shape) != 2 or points.shape[0] not in [3, 4]:
        raise Exception("Input points should be (3,N) or (4,N) in shape, found {}".format(points.shape))

    tm = create_transformation_matrix_to_voxel_space(shape, voxel_size, (0, 0, z_offset))
    p = transform_points(points, tm) #(3, N)
    return p

def create_voxel_pointcloud(points, shape, voxel_size=(0.5,0.5,1), z_offset=0):

    points_voxel_coords = car_to_voxel_coords(points.copy(), shape, voxel_size, z_offset) #(3,N)
    points_voxel_coords = points_voxel_coords[:3].transpose(1,0) #(N, 3)
    points_voxel_coords = np.int0(points_voxel_coords)
    bev = np.zeros(shape, dtype=np.float32)
    bev_shape = np.array(shape)
    within_bounds = (np.all(points_voxel_coords >= 0, axis=1) * np.all(
        points_voxel_coords < bev_shape, axis=1))
    points_voxel_coords = points_voxel_coords[within_bounds]
    coord, count = np.unique(points_voxel_coords, axis=0, return_counts=True)
    # Note X and Y are flipped:
    bev[coord[:,1], coord[:,0], coord[:,2]] = count
    
    return bev

def normalize_voxel_intensities(bev, max_intensity=16):
    return (bev/max_intensity).clip(0,1)

def move_boxes_to_car_space(boxes, ego_pose):
    """
    Move boxes from world space to car space.
    Note: mutates input boxes.
    """
    translation = -np.array(ego_pose['translation'])
    rotation = Quaternion(ego_pose['rotation']).inverse
    for box in boxes:
        # Bring box to car space
        box.translate(translation)
        box.rotate(rotation)
        
def scale_boxes(boxes, factor):
    """
    Note: mutates input boxes
    """
    for box in boxes:
        box.wlh = box.wlh * factor

def draw_boxes(im, voxel_size, boxes, classes, z_offset=0.0):
    for box in boxes:
        # We only care about the bottom corners
        corners = box.bottom_corners()
        corners_voxel = car_to_voxel_coords(corners, im.shape, voxel_size, z_offset).transpose(1,0)
        corners_voxel = corners_voxel[:,:2] # Drop z coord
        class_color = classes.index(box.name) + 1
        if class_color == 0:
            raise Exception("Unknown class: {}".format(box.name))
        cv2.drawContours(im, np.int0([corners_voxel]), 0, (class_color, class_color, class_color), -1)

In [4]:
class BEVImageDataset(torch.utils.data.Dataset):
    def __init__(self, input_filepaths, target_filepaths, map_filepaths=None):
        self.input_filepaths = input_filepaths
        self.target_filepaths = target_filepaths
        self.map_filepaths = map_filepaths
        
        if map_filepaths is not None:
            assert len(input_filepaths) == len(map_filepaths)        
        assert len(input_filepaths) == len(target_filepaths)

    def __len__(self):
        return len(self.input_filepaths)

    def __getitem__(self, idx):
        input_filepath = self.input_filepaths[idx]
        target_filepath = self.target_filepaths[idx]        
        sample_token = input_filepath.split("/")[-1].replace("_input.png","")        
        im = cv2.imread(input_filepath, cv2.IMREAD_UNCHANGED)        
        if self.map_filepaths:
            map_filepath = self.map_filepaths[idx]
            map_im = cv2.imread(map_filepath, cv2.IMREAD_UNCHANGED)
            im = np.concatenate((im, map_im), axis=2)        
        target = cv2.imread(target_filepath, cv2.IMREAD_UNCHANGED)        
        im = im.astype(np.float32)/255
        target = target.astype(np.int64)        
        im = torch.from_numpy(im.transpose(2,0,1))
        target = torch.from_numpy(target)      
        return im, target, sample_token
    
class UNet(nn.Module):
    def __init__(
        self,
        in_channels=1,
        n_classes=2,
        depth=5,
        wf=6,
        padding=False,
        batch_norm=False,
        up_mode='upconv',
    ):
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                UNetConvBlock(prev_channels, 2 ** (wf + i), padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.up_path = nn.ModuleList()
        self.dropout = torch.nn.Dropout(0.5)
        
        for i in reversed(range(depth - 1)):
            self.up_path.append(
                UNetUpBlock(prev_channels, 2 ** (wf + i), up_mode, padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1) # dropout is absent

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i == len(self.down_path) - 1:
                x = self.dropout(x)
                continue
            elif i == len(self.down_path) - 2:
                x = self.dropout(x)
                blocks.append(x)
                x = F.max_pool2d(x, 2)
            else:
                blocks.append(x)
                x = F.max_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        return self.last(x)

class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out

class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_size, out_size, kernel_size=1),
            )

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)
        return out

## Split Training and Validation Dataset

In [6]:
level5data = LyftDataset(data_path='.', json_path='/home/bob/data/lyft_data/train_data', verbose=True)
records = [(level5data.get('sample', record['first_sample_token'])['timestamp'], record) for record in
        level5data.scene]
entries = []
for start_time, record in sorted(records):
    start_time = level5data.get('sample', record['first_sample_token'])['timestamp'] / 1000000

    token = record['token']
    name = record['name']
    date = datetime.utcfromtimestamp(start_time)
    host = "-".join(record['name'].split("-")[:2])
    first_sample_token = record["first_sample_token"]
    entries.append((host, name, date, token, first_sample_token))
df = pd.DataFrame(entries, columns=["host", "scene_name", "date", "scene_token", "first_sample_token"])

validation_hosts = ["host-a007", "host-a008", "host-a009"]
validation_df = df[df["host"].isin(validation_hosts)]
vi = validation_df.index
train_df = df[~df.index.isin(vi)]
classes = ["car", "motorcycle", "bus", "bicycle", "truck", "pedestrian", 
           "other_vehicle", "animal", "emergency_vehicle"]

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 7.5 seconds.
Reverse indexing ...
Done reverse indexing in 3.3 seconds.


## HyperParameters

In [7]:
# Some hyperparameters we'll need to define for the system
voxel_size = (0.3, 0.3, 1.0)
z_offset = -2.0
bev_shape = (496, 496, 3)
# We scale down each box so they are more separated when projected into our coarse voxel space.
box_scale = 0.9

batch_size = 2
epochs = 15
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_weights = torch.from_numpy(np.array([0.2] + [1.0]*len(classes), dtype=np.float32))
class_weights = class_weights.to(device)

True


## Load Data

In [8]:
train_data_folder = os.path.join(ARTIFACTS_FOLDER, "bev_train_data")
input_filepaths = sorted(glob.glob(os.path.join(train_data_folder, "*_input.png")))
target_filepaths = sorted(glob.glob(os.path.join(train_data_folder, "*_target.png")))
#map_filepaths = sorted(glob.glob(os.path.join(train_data_folder, "*_map.png")))
train_dataset = BEVImageDataset(input_filepaths, target_filepaths)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())

validation_data_folder = os.path.join(ARTIFACTS_FOLDER, "./bev_validation_data")
val_input_filepaths = sorted(glob.glob(os.path.join(validation_data_folder, "*_input.png")))
val_target_filepaths = sorted(glob.glob(os.path.join(validation_data_folder, "*_target.png")))
#map_filepaths = sorted(glob.glob(os.path.join(train_data_folder, "*_map.png")))
val_dataset = BEVImageDataset(val_input_filepaths, val_target_filepaths)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())

In [9]:
def get_unet_model(in_channels=6, num_output_classes=2):
    model = UNet(in_channels=in_channels, n_classes=num_output_classes, wf=6, depth=5, padding=True, 
                 up_mode='upsample')
    return model

## Train & Val

In [10]:
epochs = 50
#model = get_unet_model(in_channels=3, num_output_classes=len(classes)+1)
model = DepthCompletionNet(num_out_channels=len(classes)+1)
model = model.to(device)
#pretrained_filename = "unet/unet_epoch_15.pth"
#pretrained_filepath = os.path.join(ARTIFACTS_FOLDER, pretrained_filename)
#model.load_state_dict(torch.load(pretrained_filepath))
optim = torch.optim.Adam(model.parameters(), lr = 1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.7) #optimizer, gamma, last_epoch
train_losses = []
val_losses = []
os.makedirs('./artifacts/resnet34_496', exist_ok=True)

for epoch in range(1, epochs+1):
    ## Train
    print("Epoch", epoch)
    train_epoch_losses = []
    train_bar = tqdm_notebook(trainloader)
    model.train()
    scheduler.step()
    for ii, (X, target, sample_ids) in enumerate(train_bar):
        X = X.to(device)  # [N, 3, H, W]
        target = target.to(device)  # [N, H, W] with class indices (0, 1)
        prediction = model(X)  # [N, 2, H, W]
        loss = F.cross_entropy(prediction, target, weight=class_weights)
        optim.zero_grad()
        loss.backward()
        optim.step()        
        train_epoch_losses.append(loss.detach().cpu().numpy())
        #if ii == 0:            
         #   visualize_predictions(X, prediction, target)    
    train_loss = np.mean(train_epoch_losses)
    train_losses.append(train_loss)
    print("Train Loss:", train_loss)
    checkpoint_filename = "resnet34_496/resnet_epoch_{}.pth".format(epoch)
    checkpoint_filepath = os.path.join(ARTIFACTS_FOLDER, checkpoint_filename)
    torch.save(model.state_dict(), checkpoint_filepath)
    print(checkpoint_filepath)
    
    ## Val
    val_epoch_losses = []
    val_bar = tqdm_notebook(valloader)
    with torch.no_grad():
        model.eval()
        for ii, (val_X, val_target, batch_sample_tokens) in enumerate(val_bar):
            val_X = val_X.to(device)  # [N, 1, H, W]
            val_target = val_target.to(device)  # [N, H, W] with class indices (0, 1)
            prediction = model(X)  # [N, 2, H, W]
            loss = F.cross_entropy(prediction, target, weight=class_weights)
            val_epoch_losses.append(loss.detach().cpu().numpy())
    val_loss = np.mean(val_epoch_losses)
    val_losses.append(val_loss)
    print("Val loss:", np.mean(val_loss))
x = range(1, epochs+1)
plt.plot(x, train_losses, 'bo', linestyle='dashed', linewidth=2, markersize=12)
plt.plot(x, val_losses, 'g*', linestyle='dashed', linewidth=2, markersize=12)
plt.legend(['train', 'val'])

NameError: name 'resnet' is not defined

## mAP on Val

In [9]:
import json
import gc
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision

def val_map(epoch):
    ## load model
    model = get_unet_model(in_channels=3, num_output_classes=1+len(classes))
    model = model.to(device)
    checkpoint_filename = "resnet34_496/resnet_epoch_{}.pth".format(epoch)
    checkpoint_filepath = os.path.join(ARTIFACTS_FOLDER, checkpoint_filename)
    model.load_state_dict(torch.load(checkpoint_filepath))
    
    gc.collect()
    progress_bar = tqdm_notebook(valloader)
    targets = np.zeros((len(val_target_filepaths), bev_shape[0], bev_shape[1]), dtype=np.uint8)
    # We quantize to uint8 here to conserve memory. We're allocating >20GB of memory otherwise.
    predictions = np.zeros((len(val_target_filepaths), 1+len(classes), bev_shape[0], 
                            bev_shape[1]), dtype=np.uint8)
    sample_tokens = []
    
    ## predict
    with torch.no_grad():
        model.eval()
        for ii, (X, target, batch_sample_tokens) in enumerate(progress_bar):
            offset = ii*batch_size
            targets[offset:offset+batch_size] = target.numpy()
            sample_tokens.extend(batch_sample_tokens)
            X = X.to(device)  # [N, 1, H, W]
            target = target.to(device)  # [N, H, W] with class indices (0, 1)
            prediction = model(X)  # [N, 2, H, W]
            prediction = F.softmax(prediction, dim=1)
            prediction_cpu = prediction.cpu().numpy()
            predictions[offset:offset+batch_size] = np.round(prediction_cpu*255).astype(np.uint8)
    predictions_non_class0 = 255 - predictions[:,0]   
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    predictions_opened = np.zeros((predictions_non_class0.shape), dtype=np.uint8)
    background_threshold = 255//2
    for i, p in enumerate(tqdm(predictions_non_class0)):
        thresholded_p = (p > background_threshold).astype(np.uint8)
        predictions_opened[i] = cv2.morphologyEx(thresholded_p, cv2.MORPH_OPEN, kernel)
    
    ## 2d detection
    detection_boxes = []
    detection_scores = []
    detection_classes = []

    for i in tqdm_notebook(range(len(predictions))):
        prediction_opened = predictions_opened[i]
        probability_non_class0 = predictions_non_class0[i]
        class_probability = predictions[i]
        sample_boxes = []
        sample_detection_scores = []
        sample_detection_classes = []
        contours, hierarchy = cv2.findContours(prediction_opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)     
        for cnt in contours:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)        
            # Let's take the center pixel value as the confidence value
            box_center_index = np.int0(np.mean(box, axis=0))        
            for class_index in range(len(classes)):
                box_center_value = class_probability[class_index+1, box_center_index[1], box_center_index[0]]            
                # Let's remove candidates with very low probability
                if box_center_value < 0.01:
                    continue            
                box_center_class = classes[class_index]
                box_detection_score = box_center_value
                sample_detection_classes.append(box_center_class)
                sample_detection_scores.append(box_detection_score)
                sample_boxes.append(box)            
        detection_boxes.append(np.array(sample_boxes))
        detection_scores.append(sample_detection_scores)
        detection_classes.append(sample_detection_classes)
    
    ## 3d detection
    pred_box3ds = []
    for (sample_token, sample_boxes, sample_detection_scores, sample_detection_class) in tqdm_notebook(
            zip(sample_tokens, detection_boxes, detection_scores, 
            detection_classes), total=len(sample_tokens)):
        sample_boxes = sample_boxes.reshape(-1, 2) # (N, 4, 2) -> (N*4, 2)
        sample_boxes = sample_boxes.transpose(1,0) # (N*4, 2) -> (2, N*4)
        # Add Z dimension
        sample_boxes = np.vstack((sample_boxes, np.zeros(sample_boxes.shape[1]),)) # (2, N*4) -> (3, N*4)
        sample = level5data.get("sample", sample_token)
        sample_lidar_token = sample["data"]["LIDAR_TOP"]
        lidar_data = level5data.get("sample_data", sample_lidar_token)
        lidar_filepath = level5data.get_sample_data_path(sample_lidar_token)
        ego_pose = level5data.get("ego_pose", lidar_data["ego_pose_token"])
        ego_translation = np.array(ego_pose['translation'])
        global_from_car = transform_matrix(ego_pose['translation'],
                                       Quaternion(ego_pose['rotation']), inverse=False)
        car_from_voxel = np.linalg.inv(create_transformation_matrix_to_voxel_space(
                                        bev_shape, voxel_size, (0, 0, z_offset)))
        global_from_voxel = np.dot(global_from_car, car_from_voxel)
        sample_boxes = transform_points(sample_boxes, global_from_voxel)
        sample_boxes[2,:] = ego_pose["translation"][2]
        # (3, N*4) -> (N, 4, 3)
        sample_boxes = sample_boxes.transpose(1,0).reshape(-1, 4, 3)
        # We don't know the height of our boxes, let's assume every object is the same height.
        box_height = 1.75
        sample_boxes_centers = sample_boxes.mean(axis=1)
        sample_boxes_centers[:,2] += box_height/2    
        sample_lengths = np.linalg.norm(sample_boxes[:,0,:] - sample_boxes[:,1,:], axis=1) * 1/box_scale
        sample_widths = np.linalg.norm(sample_boxes[:,1,:] - sample_boxes[:,2,:], axis=1) * 1/box_scale    
        sample_boxes_dimensions = np.zeros_like(sample_boxes_centers) 
        sample_boxes_dimensions[:,0] = sample_widths
        sample_boxes_dimensions[:,1] = sample_lengths
        sample_boxes_dimensions[:,2] = box_height
        for i in range(len(sample_boxes)):
            translation = sample_boxes_centers[i]
            size = sample_boxes_dimensions[i]
            class_name = sample_detection_class[i]
            ego_distance = float(np.linalg.norm(ego_translation - translation))          
            # Determine the rotation of the box
            v = (sample_boxes[i,0] - sample_boxes[i,1])
            v /= np.linalg.norm(v)
            r = R.from_dcm([ # initialization from direction cosine matrices
                [v[0], -v[1], 0],
                [v[1],  v[0], 0],
                [   0,     0, 1],
            ])
            quat = r.as_quat() # as quaternions
            # XYZW -> WXYZ order of elements
            quat = quat[[3,0,1,2]]        
            detection_score = float(sample_detection_scores[i])        
            box3d = Box3D(
                sample_token=sample_token,
                translation=list(translation),
                size=list(size),
                rotation=list(quat),
                name=class_name,
                score=detection_score
            )
            pred_box3ds.append(box3d)
    
    ## mAP calc
    pred = [b.serialize() for b in pred_box3ds]
    with open(os.path.join(ARTIFACTS_FOLDER, "json/unet500/pred_{}.json".format(epoch)), "w") as f:
        json.dump(pred, f)
    

In [10]:
val_map(epoch=12)
val_map(epoch=13)

/home/bob/anaconda3/envs/torch37/lib/python3.7/site-packages/torch/nn/functional.py:2539: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


100%|██████████| 5040/5040 [00:01<00:00, 4235.28it/s]


100%|██████████| 5040/5040 [00:01<00:00, 4025.14it/s]


In [11]:
gc.collect()

52

In [18]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet500/pred_11.json' --gt/home/bob/code/lyft/eval_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.1050620193022741
('animal', 0.0)
('bicycle', 0.03192542878666849)
('bus', 0.051367276695549226)
('car', 0.485934698561018)
('motorcycle', 0.012984378170014202)
('other_vehicle', 0.24332543008689356)
('pedestrian', 0.007177650093509974)
('truck', 0.007781292024539437)


In [20]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet500/pred_10.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.10303964494351976
('animal', 0.0)
('bicycle', 0.020240364505013236)
('bus', 0.05615950939036278)
('car', 0.4896918134879935)
('motorcycle', 0.002967682212965232)
('other_vehicle', 0.24063487119444624)
('pedestrian', 0.006619156133551654)
('truck', 0.008003762623825527)


In [12]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet500/pred_12.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.09696393948890356
('animal', 0.0)
('bicycle', 0.020522331424150303)
('bus', 0.044108999504336756)
('car', 0.4760304025770559)
('motorcycle', 0.00936083077373117)
('other_vehicle', 0.21244530272431872)
('pedestrian', 0.006776530621727134)
('truck', 0.0064671182859085494)


In [13]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet500/pred_13.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.106139026940494
('animal', 0.0)
('bicycle', 0.03194584508237589)
('bus', 0.05485744524089356)
('car', 0.4819533836688439)
('motorcycle', 0.009448588562234898)
('other_vehicle', 0.2553146374039543)
('pedestrian', 0.0084616114672506)
('truck', 0.0071307040983989)


In [18]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_7.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.09914757147483642
('animal', 0.0)
('bicycle', 0.011524515325533953)
('bus', 0.06473120511491712)
('car', 0.47667239016310103)
('motorcycle', 0.004528301886792452)
('other_vehicle', 0.2261457226031809)
('pedestrian', 0.0029124416545519916)
('truck', 0.006665995050613903)


In [19]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_8.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.10267539615864357
('animal', 0.0)
('bicycle', 0.0049842166323256325)
('bus', 0.06498048608258282)
('car', 0.4997703019744049)
('motorcycle', 0.005302449250907603)
('other_vehicle', 0.23059450862261271)
('pedestrian', 0.0034707821531930003)
('truck', 0.012300424553121834)


In [20]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_9.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.10058817970429339
('animal', 0.0)
('bicycle', 0.00501539544602367)
('bus', 0.0661227842668923)
('car', 0.4812704256633357)
('motorcycle', 0.0014483127617571632)
('other_vehicle', 0.23472350780008439)
('pedestrian', 0.004214098687682446)
('truck', 0.011910913008571404)


In [21]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_10.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.09661872671003473
('animal', 0.0)
('bicycle', 0.006207686512882227)
('bus', 0.05365747843992068)
('car', 0.4625923395543592)
('motorcycle', 0.0013331993995301156)
('other_vehicle', 0.23447758587535306)
('pedestrian', 0.002867096202367134)
('truck', 0.011814427695865532)


In [22]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_11.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.1083809834325789
('animal', 0.0)
('bicycle', 0.014377314889907412)
('bus', 0.07868036752077714)
('car', 0.4879433012496469)
('motorcycle', 0.0036513035763962283)
('other_vehicle', 0.2664949674229056)
('pedestrian', 0.0045337745158624595)
('truck', 0.011366838285135477)


In [23]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_12.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.1011208595355569
('animal', 0.0)
('bicycle', 0.018045573384186407)
('bus', 0.06715809217886828)
('car', 0.4721368518106879)
('motorcycle', 0.0037810721772985924)
('other_vehicle', 0.23084642142279588)
('pedestrian', 0.0045671306990924)
('truck', 0.012431734611525764)


In [25]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_13.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.1073798958101321
('animal', 0.0)
('bicycle', 0.009746183221966291)
('bus', 0.0753229748620661)
('car', 0.4861778906690318)
('motorcycle', 0.0016264120037704944)
('other_vehicle', 0.2695621634185528)
('pedestrian', 0.004223498539495018)
('truck', 0.012380043766174222)


In [26]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_14.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.10645109231341443
('animal', 0.0)
('bicycle', 0.01385349517747148)
('bus', 0.08394761171372919)
('car', 0.463558322179128)
('motorcycle', 0.02210876986253622)
('other_vehicle', 0.25726352624599613)
('pedestrian', 0.003355396881664816)
('truck', 0.0075216164467896635)


In [27]:
!python ./eval/mAP_evaluation.py --pred_file './artifacts/json/unet/pred_15.json' --gt_file "./artifacts/gt.json" --iou_threshold 0.4

Class_names =  ['animal', 'bicycle', 'bus', 'car', 'motorcycle', 'other_vehicle', 'pedestrian', 'truck']
Average per class mean average precision =  0.1078577617762986
('animal', 0.0)
('bicycle', 0.013198872017253593)
('bus', 0.07745053242650779)
('car', 0.4916629343578503)
('motorcycle', 0.002830188679245283)
('other_vehicle', 0.2622120155137613)
('pedestrian', 0.0032572098740683572)
('truck', 0.012250341341702343)
